In [8]:
%reload_ext autoreload
%autoreload 2

from Preprocessor import *
from CNNDiscriminator import *

%aimport Preprocessor
%aimport CNNDiscriminator

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, models, transforms
import torch.nn.functional as F
import time
import pandas as pd
import os
import copy
from matplotlib import pyplot as plt
import re
import string
import nltk

In [9]:
class MyDataset(Dataset):
    def __init__(self, values, labels):
        super(MyDataset, self).__init__()
        self.values = values
        self.labels = labels

    def __len__(self):
        return len(self.values)

    def __getitem__(self, index):
        return self.values[index], self.labels[index]

In [11]:
discriminator = CNNDiscriminator.CNNDiscriminator()
test = torch.randint(low=0,high=50,size=(8,5)) # 8 sentences of 5 words
display(test)
a = discriminator(test)
display(a)

tensor([[ 8, 40,  0, 28, 45],
        [29, 26, 31, 33,  3],
        [48, 28, 15, 29, 35],
        [18, 16, 24, 42, 27],
        [27, 48, 17,  3, 25],
        [21, 11, 30, 35,  1],
        [46, 39, 25, 30, 44],
        [15, 36, 45, 10, 31]])

tensor([[6.5220e-01, 3.4780e-01],
        [2.9920e-01, 7.0080e-01],
        [3.8517e-01, 6.1483e-01],
        [1.0118e-02, 9.8988e-01],
        [9.9995e-01, 4.8044e-05],
        [1.0974e-03, 9.9890e-01],
        [4.7708e-06, 1.0000e+00],
        [9.9590e-01, 4.1050e-03]], grad_fn=<SoftmaxBackward>)

In [12]:
# Training part : features needed
num_epochs = 1
num_classes = 2
batch_size = 8
learning_rate = 0.001

criterion = nn.BCELoss() # nn.BCEWithLogitsLoss stabler ? 

# Establish convention for real and fake labels during training
real_label = 1
fake_label = 0

# Beta1 hyperparam for Adam optimizers
# beta1 = 0.5
# optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer = torch.optim.Adam(discriminator.parameters(), lr=learning_rate)

# Create a batch of data
samples = torch.randint(low=0,high=50,size=(400,5))
labels = torch.randint(low=0,high=2,size=(samples.size(0),1)).float() 
train_dataset = MyDataset(samples, labels)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [13]:
# Training
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    
    for i, (sentences, labels) in enumerate(train_loader):
        
        # Run the forward pass
        outputs = discriminator(sentences)
        true_probability = outputs[:,1].unsqueeze(1)
        loss = criterion(true_probability, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data,1)
        predicted = predicted.unsqueeze(1)
        correct = (predicted == labels).sum().item() 
        acc_list.append(correct / total)
        
        # Print the results 
        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/1], Step [1/50], Loss: 2.5218, Accuracy: 37.50%
Epoch [1/1], Step [2/50], Loss: 1.9235, Accuracy: 50.00%
Epoch [1/1], Step [3/50], Loss: 5.8824, Accuracy: 50.00%
Epoch [1/1], Step [4/50], Loss: 4.2460, Accuracy: 12.50%
Epoch [1/1], Step [5/50], Loss: 0.9108, Accuracy: 62.50%
Epoch [1/1], Step [6/50], Loss: 5.5763, Accuracy: 37.50%
Epoch [1/1], Step [7/50], Loss: 6.4622, Accuracy: 25.00%
Epoch [1/1], Step [8/50], Loss: 1.3533, Accuracy: 50.00%
Epoch [1/1], Step [9/50], Loss: 3.9195, Accuracy: 25.00%
Epoch [1/1], Step [10/50], Loss: 4.0677, Accuracy: 37.50%
Epoch [1/1], Step [11/50], Loss: 1.9772, Accuracy: 37.50%
Epoch [1/1], Step [12/50], Loss: 4.2313, Accuracy: 50.00%
Epoch [1/1], Step [13/50], Loss: 3.8190, Accuracy: 25.00%
Epoch [1/1], Step [14/50], Loss: 3.8073, Accuracy: 50.00%
Epoch [1/1], Step [15/50], Loss: 3.3673, Accuracy: 62.50%
Epoch [1/1], Step [16/50], Loss: 2.9522, Accuracy: 25.00%
Epoch [1/1], Step [17/50], Loss: 3.4903, Accuracy: 62.50%
Epoch [1/1], Step [18/5

In [16]:
preprocessor = Preprocessor.Preprocessor()
preprocessor.load_dataset('data.xls') # Load the dataset
output_sentences, output_descriptions = preprocessor.preprocess()

In [17]:
display(output_descriptions.shape, output_sentences.shape)
# display(d[0], s[0])

vocabulary = preprocessor.vocabulary
display(len(vocabulary))

desc_original = preprocessor.descriptions_original
# display(desc_original[9])

desc_tokened = preprocessor.descriptions_tokened
sent_tokened = preprocessor.sentences_tokened
# display(desc_tokened[9])

desc_numeric = preprocessor.descriptions_indexed
sent_numeric = preprocessor.sentences_indexed
# display(desc_numeric[9])

torch.Size([2206, 998])

torch.Size([35555, 25])

25254

In [ ]:
# Test divers size distribution avec desc_numeric or sent_numeric
display(len(desc_numeric))

lens = [len(d) for d in desc_numeric]
plt.hist(lens, bins=50)
plt.gca().set(title='Frequency Histogram', ylabel='Frequency');
# u,c = np.unique(lens, return_counts=True)
# display(u,c)

lens_filtered = [l for l in lens if 15 <= l <= 900]
display(len(lens_filtered))
plt.hist(lens_filtered, bins=50)
plt.gca().set(title='Frequency Histogram', ylabel='Frequency');

In [ ]:
# Quick training test for the text
samples = output_sentences
labels = torch.randint(low=0,high=2,size=(samples.size(0),1)).float() 
train_dataset = MyDataset(samples, labels)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
discri_test2 = CNNDiscriminator(voc_size=len(vocabulary)+1)

total_step = len(train_loader)
loss_list = []
acc_list = []

for epoch in range(num_epochs):
    
    for i, (sentences, labels) in enumerate(train_loader):
        
        # Run the forward pass
        outputs = discri_test2(sentences)
        true_probability = outputs[:,1].unsqueeze(1)
        loss = criterion(true_probability, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data,1)
        predicted = predicted.unsqueeze(1)
        correct = (predicted == labels).sum().item() 
        acc_list.append(correct / total)
        
        # Print the results 
        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))